# Creación Fact Ventas

In [0]:
from pyspark.sql.functions import col, current_timestamp, coalesce, lit

# Carga de tablas
df_ventas = spark.read.table("workspace.silver.ventas")
dim_prod = spark.read.table("workspace.gold.dim_productos")
dim_cli = spark.read.table("workspace.gold.dim_clientes")
dim_tiempo = spark.read.table("workspace.gold.dim_tiempo")

# 1. Definimos una clave por defecto
DEFAULT_KEY = -1 

fact_ventas = df_ventas.join(dim_prod, df_ventas.codigo_producto == dim_prod.codigo_producto_bk, "left") \
    .join(dim_cli, df_ventas.numero_identificacion == dim_cli.identificacion_bk, "left") \
    .join(dim_tiempo, df_ventas.fecha == dim_tiempo.fecha_bk, "left") \
    .select(
        # Si el join falló y la SK es nula, ponemos nuestra DEFAULT_KEY
        coalesce(col("tiempo_key"), lit(DEFAULT_KEY)).alias("tiempo_key"),
        coalesce(col("producto_key"), lit(DEFAULT_KEY)).alias("producto_key"),
        coalesce(col("cliente_key"), lit(DEFAULT_KEY)).alias("cliente_key"),
        
        col("cantidad"),
        col("precio_unitario"),
        col("subtotal"),
        col("iva"),
        col("total"),
        current_timestamp().alias("fecha_carga")
    )

fact_ventas.write.mode("overwrite").saveAsTable("workspace.gold.fact_ventas")
print("✅ fact_ventas creada.")